In [1]:
import os
from pathlib import Path


In [8]:
# 0a43caca-ddb9-4041-8954-213dfbeb1150_capped.jpg

# delete all images that are "...origin.jpg"
folder_path = Path(r'C:\Users\xx\Desktop\Workspace\selfcurenetwork-own\test\face')

for folder_name in os.scandir(folder_path):
    print(folder_name)
    for filename in os.listdir(folder_name):
        if filename.endswith("origin.jpg"):
            os.remove(os.path.join(folder_path, folder_name ,filename))
        

<DirEntry 'angry'>
<DirEntry 'disgust'>
<DirEntry 'fear'>
<DirEntry 'happy'>
<DirEntry 'neutral'>
<DirEntry 'sad'>
<DirEntry 'surprise'>
<DirEntry 'undetected'>


### Relabelling

In [5]:
from pigeonXT import annotate
from IPython.display import display, Image
from fastai.vision.all import get_image_files
from pathlib import Path


image_filenames = get_image_files(Path(r"C:\Users\xx\Desktop\Workspace\selfcurenetwork-own\NTU-Data\data"), folders="sad")
emotions = "surprise, fear, disgust, happy, sad, angry, neutral".split(", ")
def display_fn(filename):
    print(f"Original label - {filename.parent.name}")
    display(Image(filename))
annotations = annotate(image_filenames,
options=emotions, display_fn=display_fn, shuffle=True, include_back=True)

HTML(value='0 of 3164 Examples annotated, Current Position: 0 ')

Output()

In [7]:
len(annotations)

12

In [8]:
anno_str = {}
for key, value in annotations.items():
    anno_str[str(key)] = value
anno_str

{'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\38a9f250-cf0b-44cf-ad0e-74f288b491c4_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\631d3f66-c800-4d7b-8485-32c404afe33e_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\aae9ed1f-32f4-4e69-98f6-0e8028fd2931_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\1d6abcea-ffae-4556-ab23-412432818e3d_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\1bf4bcc5-ff9a-46f0-bf6d-6fc8ffeba437_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\7a21fbc4-f780-4c73-abcb-01b74782313b_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\ceae2116-5c41-4e64-a572-f2b4fe616a31_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenet

In [ ]:
# master_dict = {}

In [16]:
master_dict={**master_dict, **anno_str}
master_dict

{'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\38a9f250-cf0b-44cf-ad0e-74f288b491c4_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\631d3f66-c800-4d7b-8485-32c404afe33e_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\aae9ed1f-32f4-4e69-98f6-0e8028fd2931_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\1d6abcea-ffae-4556-ab23-412432818e3d_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\1bf4bcc5-ff9a-46f0-bf6d-6fc8ffeba437_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\7a21fbc4-f780-4c73-abcb-01b74782313b_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenetwork-own\\NTU-Data\\data\\sad\\ceae2116-5c41-4e64-a572-f2b4fe616a31_capped.jpg': 'sad',
 'C:\\Users\\xx\\Desktop\\Workspace\\selfcurenet

In [13]:
len(master_dict)

0

In [15]:
# Save json
import json
with open("anno.json", "w") as outfile:  
    json.dump(master_dict, outfile) 

{}

In [84]:
# Load json and analyze
import pandas as pd

with open('anno.json') as json_file:
    data = json.load(json_file)
    df = pd.DataFrame.from_dict(data, orient="index", columns=["emotion"])
df = df.reset_index()
df.groupby(["emotion"]).emotion.value_counts()

emotion   emotion 
angry     angry       128
disgust   disgust     139
fear      fear        117
happy     happy       380
neutral   neutral     165
sad       sad         156
surprise  surprise    105
Name: emotion, dtype: int64

In [17]:
folder = Path(r"C:\Users\xx\Desktop\Workspace\selfcurenetwork-own\NTU-Data\manual-relabel")

In [18]:
# Copy cleaned files into new folder
import shutil
copy_dict = master_dict.copy()
for key,value in copy_dict.items():
    src = Path(key)
    if not src.exists():
        master_dict.pop(key)
        continue
    dest_folder = folder/value
    dest_folder.mkdir(parents=True, exist_ok=True)

    shutil.copy(src, dest_folder/src.name)

### Remove duplicate images recursively

In [2]:
from PIL import Image
import imagehash

image_fns = get_image_files(Path(r"C:\Users\xx\Desktop\Workspace\selfcurenetwork-own\NTU-Data\manual-relabel\sad"))


In [3]:
img_hashes = {}

count = 0
for image_fn in sorted(image_fns):
    hash = imagehash.average_hash(Image.open(image_fn))
    if hash in img_hashes:
        print( '{} duplicate of {}'.format(image_fn, img_hashes[hash]) )
        count += 1
        image_fn.unlink()
    else:
        img_hashes[hash] = image_fn

In [4]:
count

0